In [1]:
# setups

import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session


In [3]:
sess = Session()

role = get_execution_role()

%store -r pt_mnist_model_data

# Load in trained model from s3 bucket
try:
    pt_mnist_model_data
except NameError:
    import json

    # copy a pretrained model from a public public to your default bucket
    s3 = boto3.client("s3")
    bucket = CONFIG["public_bucket"]
    key = "test/pytorch-training-2023-02-19-18-16-23-445/model.tar.gz"
    s3.download_file(bucket, key, "model.tar.gz")

    # upload to default bucket
    pt_mnist_model_data = sess.upload_data(
        path="model.tar.gz", bucket=sess.default_bucket(), key_prefix="model/pytorch"
    )
print(pt_mnist_model_data)

s3://sagemaker-us-east-1-472925017889/test/pytorch-training-2023-02-19-18-16-23-445/model.tar.gz


In [5]:
model = PyTorchModel(
    entry_point="train.py",
    source_dir="code",
    role=role,
    model_data=pt_mnist_model_data,
    framework_version="1.5.0",
    py_version="py3",
)

In [7]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# Deploy the model
instance_type = "local"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Attaching to crhfmegqhy-algo-1-hwq1z
crhfmegqhy-algo-1-hwq1z | 2023-02-19 18:57:29,968 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
crhfmegqhy-algo-1-hwq1z | MMS Home: /opt/conda/lib/python3.6/site-packages
crhfmegqhy-algo-1-hwq1z | Current directory: /
crhfmegqhy-algo-1-hwq1z | Temp directory: /home/model-server/tmp
crhfmegqhy-algo-1-hwq1z | Number of GPUs: 0
crhfmegqhy-algo-1-hwq1z | Number of CPUs: 2
crhfmegqhy-algo-1-hwq1z | Max heap size: 859 M
crhfmegqhy-algo-1-hwq1z | Python executable: /opt/conda/bin/python3.6
crhfmegqhy-algo-1-hwq1z | Config file: /etc/sagemaker-mms.properties
crhfmegqhy-algo-1-hwq1z | Inference address: http://0.0.0.0:8080
crhfmegqhy-algo-1-hwq1z | Management address: http://0.0.0.0:8080
crhfmegqhy-algo-1-hwq1z | Model Store: /.sagemaker/mms/models
crhfmegqhy-algo-1-hwq1z | Initial Models: ALL
crhfmegqhy-algo-1-hwq1z | Log dir: /logs
crhfmegqhy-algo-1-hwq1z | Metrics dir: /logs
crhfmegqhy-algo-1-hwq1z | Netty threads: 0
crhfmegqhy-algo-1-h

In [ ]:
dummy_data = {"UserID": "5", "num_recs": "3"}
res = predictor.predict({"inputs" : dummy_data})
print(res)

TypeError: unhashable type: 'dict'